# script used to estimate model performance on benchmark w/ a linear readout

# environment setup

In [ ]:
import torch,  pickle, pandas, os, json
import numpy as np
from PIL import Image
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from scipy import stats
import tqdm as notebook_tqdm
from thop import profile

# download mochi benchmark from huggingface

In [ ]:
from datasets import load_dataset

# download huggingface dataset 
mochi = load_dataset("tzler/MOCHI")['train']

# load dataframe from your local path

In [ ]:
df = pandas.read_csv("/datasets/hida/current/neurips_benchmark/benchmark.csv")
df.rename(columns={'Unnamed: 0': 'idx'}, inplace=True)

# define common function to extract features from dinos, clips, maes

In [35]:
from transformers import AutoImageProcessor, ViTMAEModel
from transformers import AutoImageProcessor, AutoModel
import open_clip

In [36]:
class define_clip:
    """
    use: X.extract_features(imagename, modelname)
         e.g. X.extract_features(image, 'ViT-B-32')

    """
    def __init__(self, model, dataset):

        self.make = open_clip.create_model_and_transforms
        self.name = model
        self.dataset = dataset
        self.model, _, self.prep = self.make(model,pretrained=dataset)
        self.model.eval()

    def extract(self, images):
        with torch.no_grad():
            inputs = self.prep(images).unsqueeze(0)
            features = self.model.encode_image(inputs)
        return np.squeeze(features.detach().numpy())

class define_dino:

    """
    # https://huggingface.co/docs/transformers/model_doc/dinov2
    """

    def __init__(self, model):

        self.model = AutoModel.from_pretrained("facebook/%s"%model)
        self.prep = AutoImageProcessor.from_pretrained("facebook/%s"%model)
        self.model.eval()

    def extract(self, images):
        with torch.no_grad():
          inputs = self.prep(images=images, return_tensors="pt")
          features = self.model(**inputs).pooler_output.detach().numpy()
        return features

class define_mae:

    """
    extrac cls token from maes
    from: https://github.com/facebookresearch/mae/blob/main/models_mae.py#L161
    """

    def __init__(self, model):

        self.model = ViTMAEModel.from_pretrained("facebook/%s"%model)
        self.prep = AutoImageProcessor.from_pretrained("facebook/%s"%model)
        self.model.eval()


    def extract(self, images):
        with torch.no_grad():
            inputs = self.prep(images=images, return_tensors="pt")
            features = self.model(**inputs).last_hidden_state
        return features[:, 0, :].detach().numpy().squeeze()

# define functions to extract features and generate same-different training data

In [37]:
def extract_features0(df, model, mochi, single=True):

  images, oddities, features = {} , {}, []

  for index, i_trial in df.iterrows():

    oddities[i_trial.trial] = i_trial.oddity_index
    
    for i_index, i_image in enumerate( extract_list( i_trial.images ) ):
      
      # open image
      i_image = mochi[index]['images'][i_index] 
      # extract features maybe
      if single: features.append( model(i_image).flatten() )
      # change to import from huggingface
      images['%s-%s'%(i_trial.trial,i_index)] = mochi[index]['images'][i_index]

  # extract features maybe
  if not single: features = model([images[i] for i in images])

  return np.array(features).squeeze(), oddities, images

def generate_delta_vectors0(trials, features, oddities, images):

  def delta(a, b, xtype='diff'):

    if xtype == 'diff':
        x = a - b
    elif xtype == 'abs':
        x = np.array(np.abs(a - b))
    elif xtype == 'sqrt':
        x = np.sqrt(np.array(np.abs(a - b)))
    elif xtype == 'sqr':
        x = (np.array(np.abs(a - b)))**2
    elif xtype == 'product':
        x = a * b

    return x

  trial_markers = []
  deltas = []
  labels = []

  for i_trial in trials:

    trial_indices = [i.split('-')[0] == i_trial for i in images ]

    i_features = features[ trial_indices ]

    oddity_index = oddities[i_trial]

    pairs = [] # we just need once -- okay to override

    for i in range(len(i_features)):

      for j in range(i+1,len(i_features)):

        i_delta = delta(i_features[i], i_features[j])
        deltas.append( i_delta )
        trial_markers.append( i_trial )
        pairs.append( (i,j) )

        if (i == oddity_index) or (j == oddity_index):
          labels.append(0) # different
        else:
          labels.append(1) # same

  all_inds = np.array(range(len(i_features)))

  location_of_indices = {i: np.nonzero((pairs == i).mean(1))[0] for i in all_inds}

  info = {'deltas': np.array(np.squeeze(deltas)),
          'labels': np.array(labels),
          'trials':trial_markers,
          'inds': all_inds,
          'locs': location_of_indices}

  return info

def extract_list(l):
  return [i[1:-1] for i in l[1:-1].split(', ')]

# define model to evaluate

In [38]:
dinos = ['dinov2-base','dinov2-large', 'dinov2-giant']

maes = ['vit-mae-base','vit-mae-large', 'vit-mae-huge']

clips = {'ViT-B-32': 'laion2b_s34b_b79k',
         'ViT-B-16': 'laion2b_s34b_b88k',
         'ViT-L-14': 'laion2b_s32b_b82k',
         'ViT-H-14': 'laion2b_s32b_b79k',
         'ViT-g-14': 'laion2b_s12b_b42k'}

In [40]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

save_nparams = 1

In [41]:
models_to_evaluate = ['dinov2-base']

n_params = {}
models = {}
n_flops = {}

for i_model in models_to_evaluate:

  if i_model in dinos:
    _model = define_dino(i_model)
  elif i_model in maes:
    _model = define_mae(i_model)
  elif i_model in clips:
    _model = define_clip(i_model, clips[i_model])

  n_flops[i_model] =  profile(_model.model, inputs=(torch.randn(1, 3, 224, 224), ))
  n_params[i_model] = count_parameters(_model.model)

  models[i_model] = _model.extract

if save_nparams:

  with open('../assets/your_model_params_count.pkl', 'wb') as f:
    pickle.dump({'n_params': n_params, 'flops': n_flops}, f)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.


In [ ]:
# number of independent splits used to train the svm on each trial
n_permutations = 100
# fraction of data to include in each split
subset_fraction = 0.75

# iterate through each dataset
for i_dataset in df.dataset.unique():

  # iterate through each condition
  for i_condition in df[df.dataset==i_dataset].condition.unique():
    # select subset of data to train and test on
    idf = df[df.condition==i_condition]

    # identify all trials
    trials0 = idf.trial.unique()

    for i_model in list(models):

      # extract features from this dataset
      features0, oddities0, images0 = extract_features0(idf, models[i_model], mochi)

      # # # for each trial, generate vector of differences between each image
      diffs = generate_delta_vectors0(trials0, features0, oddities0, images0)

      for i_index, i_trial in idf.iterrows():

        # determine the index/image of the oddity
        i_oddity_index = oddities0[i_trial.trial]

        # extract indices and labels for training svm
        train_indices = [i.split('-')[0] != i_trial.trial for i in diffs['trials']]
        # vectors for the different between each image vector
        X_train = diffs['deltas'][train_indices,:]
        # labels for whether each vector was 'same' or 'different'
        y_train = diffs['labels'][train_indices].T

        # extract indices and labels for training svm
        test_indices = [i.split('-')[0] == i_trial.trial for i in diffs['trials']]
        # vectors for the different between each image vector
        X_test = diffs['deltas'][test_indices,:]
        # labels for whether each vector was 'same' or 'different'
        y_test = diffs['labels'][test_indices].T
        # for each iteration use X% of the available trials
        len_subset = int(subset_fraction*len(X_train))

        # prep to each each iteration
        choices = []

        for _ in range(n_permutations):

          # identify random subset of delta vectors to train on
          random_subset = np.random.permutation(len(X_train))[:len_subset]

          # define model to train a linear readout
          clf = make_pipeline(StandardScaler(),
                              SVC(class_weight='balanced', probability=True))
          # fit training data
          clf.fit(np.squeeze(X_train[random_subset,:]),y_train[random_subset])
          # predict performance on this trial
          y_hat = clf.predict_proba(X_test)

          # identify which image has the highest probability of being different
          i_diffs = [y_hat[:,0][diffs['locs'][i]].mean() for i in diffs['inds']]
          # determine whether the model-selected oddity matches ground truth
          i_trial_accuracy = i_oddity_index == np.argmax(i_diffs)
          # save
          choices.append(i_trial_accuracy)

        # store in original dataframe
        df.loc[i_trial.idx,'%s_svm_avg'%i_model] = np.mean(choices)
        df.loc[i_trial.idx,'%s_svm_med'%i_model] = np.median(choices)
        df.loc[i_trial.idx,'%s_svm_std'%i_model] = np.std(choices)
        df.loc[i_trial.idx,'%s_svm_sem'%i_model] = stats.sem(choices)
        
        # save
        df.to_csv(os.path.join(data_directory, 'your_results.csv'))